In [2]:
import sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore')

In [11]:
import numpy as np
import pandas as pd
import time
import faiss
import xarray as xr
from geopy.distance import geodesic
import geopandas as gpd
from shapely.geometry import Point
from sklearn.impute import SimpleImputer, KNNImputer
from MeteoDataset import MeteoDataset
from scipy.spatial import distance
from load_meteo_data import create_meteo_dataframe
from sklearn.neighbors import BallTree
from scipy.spatial.distance import cdist
from pathlib import Path
from constants import *
from load_era5_data import create_era5_dataframe
from eda_utils import impute_data

In [5]:
files = {'Bihar_536_Sensor_Data_Sep_2023_Screened.csv': 7, 'Bihar_536_Sensor_Data_Oct_2023_Screened.csv': 7,\
        'Bihar_536_Sensor_Data_Nov_2023_Screened.csv': 6, 'Bihar_536_Sensor_Data_Jan_2024_Screened.csv': 6,\
        'Bihar_536_Sensor_Data_Dec_2023_Screened.csv': 6, 'Bihar_512_Sensor_Data_May_Aug_Screened_Hourly.csv': 6}

nc_files = [f'{data_bihar}/era5_may_dec_2023.nc', f'{data_bihar}/era5_jan_2024.nc']

geojson_file = f'{data_bihar}/bihar.json'
pbl_file = f'{data_bihar}/PBLH_may_Dec_2023.nc'
other_params_file = f'{data_bihar}/Era5_data_May_Dec_2023.nc'
output_meteo_file = f'{data_bihar}/bihar_meteo_may_jan.pkl'
output_meteo_era5_file = f'{data_bihar}/bihar_meteo_era5_may_jan.pkl'
output_meteo_era5_imputed_file = f'{data_bihar}/bihar_meteo_era5_may_jan_knn_imputed.pkl'

region = gpd.read_file(geojson_file)
# meteo_df = pd.read_pickle(meteo_file)

start_date, end_date = pd.Timestamp('2023-05-01 00:00:00'), pd.Timestamp('2024-02-01 00:00:00')

In [6]:
if Path(output_meteo_file).is_file():
        meteo_df = pd.read_pickle(output_meteo_file)
else:
    meteo_df = create_meteo_dataframe(files, output_meteo_file, start_date, end_date)

In [7]:
min_lon, min_lat, max_lon, max_lat = region.total_bounds
min_lon, min_lat, max_lon, max_lat

(83.320238, 24.286327, 88.29954611201047, 27.521347)

In [8]:
region.total_bounds

array([83.320238  , 24.286327  , 88.29954611, 27.521347  ])

In [9]:
if Path(output_meteo_era5_file).is_file():
        era5_df = pd.read_pickle(output_meteo_era5_file)
else:
    era5_df = create_era5_dataframe(nc_files, meteo_df, output_meteo_era5_file)

In [13]:
if Path(output_meteo_era5_imputed_file).is_file():
    imputed_df = pd.read_pickle(output_meteo_era5_imputed_file)
else:
    imputed_df = impute_data(era5_df, output_meteo_era5_imputed_file, method='knn')

In [15]:
imputed_df.count()

timestamp    3384864
block        3384864
district     3384864
latitude     3384864
longitude    3384864
rh           3384864
temp         3384864
blh          3384864
u10          3384864
v10          3384864
kx           3384864
sp           3384864
tp           3384864
pm25         3384864
dtype: int64

In [16]:
df_g = imputed_df.groupby(['latitude', 'longitude'])

for loc, group in df_g:
    print(loc, group.shape)

(24.443, 84.39) (6624, 14)
(24.449, 84.581) (6624, 14)
(24.503, 85.017) (6624, 14)
(24.517, 84.668) (6624, 14)
(24.532, 84.916) (6624, 14)
(24.553, 83.811) (6624, 14)
(24.557, 86.399) (6624, 14)
(24.573, 84.803) (6624, 14)
(24.578, 85.085) (6624, 14)
(24.61, 84.147) (6624, 14)
(24.612, 84.224) (6624, 14)
(24.613, 84.669) (6624, 14)
(24.613, 86.666) (6624, 14)
(24.627, 85.212) (6624, 14)
(24.642, 85.495) (6624, 14)
(24.655, 84.588) (6624, 14)
(24.655, 85.397) (6624, 14)
(24.667, 84.785) (6624, 14)
(24.668, 84.44) (6624, 14)
(24.699, 83.613) (6624, 14)
(24.708, 85.142) (6624, 14)
(24.71476936, 84.99768066) (6624, 14)
(24.737, 85.355) (6624, 14)
(24.743, 86.728) (6624, 14)
(24.747, 86.305) (6624, 14)
(24.75, 85.556) (6624, 14)
(24.758, 84.365) (6624, 14)
(24.763, 84.982) (6624, 14)
(24.778, 86.369) (6624, 14)
(24.779, 85.687) (6624, 14)
(24.786, 85.427) (6624, 14)
(24.792, 85.007) (6624, 14)
(24.796, 84.078) (6624, 14)
(24.8, 85.032) (6624, 14)
(24.803, 85.232) (6624, 14)
(24.808, 84.788)